In [ ]:
import xarray as xr
import cv2
import xarray as xr
from netCDF4 import Dataset, num2date, date2num
#from libtiff import TIFF
import os, sys
import numpy as np

from datetime import datetime
from datetime import timedelta

import matplotlib
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation
# import cartopy.crs as ccrs
from matplotlib import cm
# from mpl_toolkits.basemap import Basemap
import warnings

warnings.filterwarnings("ignore")

from torchvision import datasets, models, transforms

In [ ]:
prcp_colours = [
                   "#FFFFFF", 
                   '#edf8b1',
                   '#c7e9b4',
                   '#7fcdbb',
                   '#41b6c4',
                   '#1d91c0',
                   '#225ea8',
                   '#253494',
                   '#4B0082',
                   "#800080",
                   '#8B0000']

prcp_colormap = matplotlib.colors.ListedColormap(prcp_colours)

levels = {}
levels["hour"]  = [0., 0.2,   1,   5,  10,  20,  30,   40,   60,   80,  100,  150]
levels["day"]   = [0., 10,  12.5,  15,  17.5,  20,  22.5,  25,  27.5,30,32.5,35,37.5,40]
levels["week"]  = [0., 0.2,  10,  20,  30,  50, 100,  150,  200,  300,  500, 1000]
levels["month"] = [0.,  10,  20,  30,  40,  50, 100,  200,  300,  500, 1000, 1500]
levels["year"]  = [0.,  50, 100, 200, 300, 400, 600, 1000, 1500, 2000, 3000, 5000]
enum={0:"0600",1:"1200",2:"1800",3:"0000",4:"0600"}

levels["crps"]   = [0., 2.,  4.,  6.,  8., 10.,  12.,  14., 16.,18.]

In [ ]:
def draw_aus(var,lat,lon,domain = [112.08, 156.25, -44.167, -10.277], level="day" ,vmin=0,vmax=1,titles_on = True, title = "CRPS", colormap = 'brg', cmap_label = "",save=False,path=""):
    """ basema_ploting .py
This function takes a 2D data set of a variable from BARRA and maps the data on miller projection. 
The map default span is longitude between 111E and 155E, and the span for latitudes is -45 to -30, this is SE Australia. 
The colour scale is YlGnBu at 11 levels. 
The levels specifed are suitable for annual rainfall totals for SE Australia. 
From the BARRA average netCDF, the mean prcp should be multiplied by 24*365
"""
#    lats.sort() #this doesn't do anything for BARRA
#    lons.sort() #this doesn't do anything for BARRA
#     domain = [111.975, 156.275, -44.525, -9.975]#awap
    from matplotlib.colors import ListedColormap, BoundaryNorm
    from mpl_toolkits.basemap import Basemap
    fig=plt.figure()
    level=levels[level]
    map = Basemap(projection = "mill", llcrnrlon = domain[0], llcrnrlat = domain[2], urcrnrlon = domain[1], urcrnrlat = domain[3], resolution = 'l')
    map.drawcoastlines()
#     map.drawmapboundary()
#     map.drawparallels(np.arange(-90., 120., 5.),labels=[1,0,0,0])
#     map.drawmeridians(np.arange(-180.,180., 5.),labels=[0,0,0,1])
    llons, llats = np.meshgrid(lon, lat) # 将维度按照 x,y 横向竖向
#     print(lon.shape,llons.shape)
    x,y = map(llons,llats)
#     print(x.shape,y.shape)
    
    norm = BoundaryNorm(level, len(level)-1)
    data=xr.DataArray(var,coords=[lat,lon],dims=["lat","lon"])
    #cs = map.pcolormesh(x, y, data, norm = norm, cmap = 'brg') 
    cs = map.pcolormesh(x, y, data, cmap = 'PiYG',vmin=vmin,vmax=vmax) 

    
    if titles_on:
        # label with title, latitude, longitude, and colormap
        
        plt.title(title)
        plt.xlabel("\n\nLongitude")
        plt.ylabel("Latitude\n\n")
        #cbar = plt.colorbar(cs,ticks = level[:-1], shrink = 0.8, extend = "max")
        cbar = plt.colorbar(cs, shrink = 1.2, extend = "max")
        cbar.ax.set_ylabel(cmap_label)
        #cbar.ax.set_xticklabels(level)
    if save:
        plt.savefig(path)
    else:
        plt.show()
    plt.cla()
    plt.close("all")
    return

In [ ]:
lp=os.listdir(r'C:\Users\123\Desktop\MMLCV1\project\crps')

avg_list=[]
x=[]
for i in range(1,31):
    filename=r'C:\Users\123\Desktop\MMLCV1\project\crps\\'+str(i)+'_50station.npy'
    c=np.load(filename)
    avg_list.append(np.array(c.mean()))
    x.append(i)
print(avg_list,x)

plt.plot(x,np.array(avg_list))
#plt.savefig(r'C:\Users\123\Desktop\crps_image\plot')
plt.xlabel('days')
plt.ylabel('crps')
plt.savefig(r'C:\Users\123\Desktop\crps_image\plot')

In [ ]:
data_r=xr.open_dataset(r'C:\Users\123\Desktop\MMLCV1\project\HRSCF_v1-main\high_r\2012-06-25.nc')

time_scope =data_r.time.values  
lat = data_r.lat.values
lon = data_r.lon.values

In [ ]:
from tqdm import tqdm
a_crpsss_my = []
a_crpsss_qm = []
a_crpsss_red = []
a_crpsss_myv2 = []

qm_mask = np.load(r"C:\Users\123\Desktop\MMLCV1\project\cacaca\qm_mask.npy")


for time in tqdm(range(1,31)):
    leading0_clima = np.load(r'C:\Users\123\Desktop\MMLCV1\project\cacaca\climatology_1\\' +"climatology_2012_all_lead_time_windows_1.npy")
    leading0_clima = leading0_clima.mean(axis=0)
    leading0_clima = np.ma.masked_array(leading0_clima,mask=qm_mask)

    leading0_qm =  np.load(r'C:\Users\123\Desktop\MMLCV1\project\cacaca\crps_qm_2006\\'+ str(time) +"_50station.npy")
    leading0_qm = leading0_qm.mean(axis=0)
    leading0_qm = np.ma.masked_array(leading0_qm,mask=qm_mask)
    
#    leading0_red =  np.load(r'C:\Users\123\Desktop\MMLCV1\project\cacaca\crps_red\\'+ str(time) +"_50station.npy")
 #   leading0_red = leading0_qm.mean(axis=0)
 #   leading0_red = np.ma.masked_array(leading0_qm,mask=qm_mask)
    
    
    

    leading0 = np.load(r'C:\Users\123\Desktop\MMLCV1\project\cacaca\crps_dem2_2006\\'+ str(time) +"_50station.npy")
    leading0 = leading0.mean(axis=0)
    leading0 = np.ma.masked_array(leading0,mask=qm_mask)
    
  






    #remove tasmania
    leading0_clima[10:100,0:886] = "nan"
    
    #right top
    leading0_clima[680:691,720:781] = "nan"
    

    #ocean -> nan
    #leading0_qm[leading0_qm==0] = -999

    #fillna
    #leading0_qm[np.isnan(leading0_qm)] = -1





    #draw_aus(leading0[date],lat=lat,lon=lon,title="my")
    #print(leading0[date].max())
    #print(leading0[date].min())

    #draw_aus(leading0_qm[date],lat=lat,lon=lon,title="QM")
    #print(leading0_qm[date].max())
    #print(leading0_qm[date].min())

    #draw_aus(leading0_clima[date],lat=lat,lon=lon,title="clima")
    #print(leading0_clima[date].max())
    #print(leading0_clima[date].min())  


    crpsss_my = np.zeros((691,886))
    crpsss_qm = np.zeros((691,886))
    crpsss_red = np.zeros((691,886))
    crpsss_v2 = np.zeros((691,886))

    for i in range(691):
        for j in range(886):
            if leading0_clima[i,j] ==0:
                crpsss_my[i,j] = 0
                #crpsss_qm[i,j] = 0
                #rpsss_v2[i,j] = 0
            else:
                crpsss_my[i,j] = 1 - (leading0[i,j]/leading0_clima[i,j])
                #crpsss_qm[i,j] = 1 - (leading0_qm[i,j]/leading0_clima[i,j])
                #crpsss_v2[i,j] = 1 - (leading0_v2[i,j]/leading0_clima[i,j])
               # crpsss_red[i,j]=1 - (leading0_red[i,j]/leading0_clima[i,j])
                crpsss_qm[i,j] = 1 - (leading0_qm[i,j]/leading0_clima[i,j])
                
#    for i in range(691):
#        for j in range(886):
#            if crpsss_my[i,j] <-100:
#                print(time)
#                print(i)
#                print(j)
                

#        for i in range(691):
#            for j in range(886):
#                if crpsss_my[date,i,j] <-1 :
#                    #print(leading0[date,i,j])
#                    #print(leading0_qm[date,i,j])
#                    #print(leading0_clima[date,i,j])
#                    crpsss_my[date,i,j] = -1

#                if crpsss_qm[date,i,j] <-1 :
#                    #print(leading0[date,i,j])
#                    #print(leading0_qm[date,i,j])
#                    #print(leading0_clima[date,i,j])
#                    crpsss_qm[date,i,j] = -1
    
    
    #filename = "F:/val09/crps_my/leading_"+ str(time) + '.png'
    draw_aus(crpsss_my,lat=lat,lon=lon,title="crpsss_my_leading_"+str(time),titles_on=True,vmin=-1,vmax=1,path=r'C:\Users\123\Desktop\crps_image\\'+str(time)+'Ynet')
    #print("my")
    #print(np.nanmax(crpsss_my))
    #print(np.nanmin(crpsss_my))
    #print(np.nanmean(crpsss_my))
    a_crpsss_my.append(np.nanmean(crpsss_my))
    print(np.nanmean(crpsss_my))
    
    #filename = "F:/val09/crps_qm/leading_"+ str(time) + '.png'
    draw_aus(crpsss_qm,lat=lat,lon=lon,title="crpsss_qm_leading_"+str(time),titles_on=True,vmin=-1,vmax=1,path=r'C:\Users\123\Desktop\crps_image\\'+str(time)+'QM')
    #print("qm")
    #print(np.nanmax(crpsss_qm))
    #print(np.nanmin(crpsss_qm))
    #print(np.nanmean(crpsss_qm))
    a_crpsss_qm.append(np.nanmean(crpsss_qm))
    print(np.nanmean(crpsss_qm))
    
   # draw_aus(crpsss_red,lat=lat,lon=lon,title="crpsss_red_leading_"+str(time),titles_on=True,vmin=-1,vmax=1,path=r'C:\Users\123\Desktop\crps_image\\'+str(time)+'QM')
    #a_crpsss_red.append(np.nanmean(crpsss_red))
    #print(np.nanmean(crpsss_red))
    
    #draw_aus(crpsss_v2,lat=lat,lon=lon,title="crpsss_myv2_leading_"+str(time),titles_on=True)
    #a_crpsss_myv2.append(np.nanmean(crpsss_v2))
    #print(np.nanmean(crpsss_v2))

    
    #filename = "F:/val12/crps_qm/leading_"+ str(time) + '.png'
    #draw_aus(crpsss_vdsrd2,lat=lat,lon=lon,title="crpsss_vdsrd2_leading_"+str(time),titles_on=True,path=filename,save=True)
    #print(np.nanmax(crpsss_vdsrd2))
    #print(np.nanmin(crpsss_vdsrd2))
    #print(np.nanmean(crpsss_vdsrd2))

In [ ]:

l1=plt.plot(x[:7],np.array(a_crpsss_my[:7]))
l2=plt.plot(x[:7],np.array(a_crpsss_qm[:7]))
#plt.savefig(r'C:\Users\123\Desktop\crps_image\plot')
plt.xlabel('days')
plt.ylabel('crps')
#plt.savefig(r'C:\Users\123\Desktop\crps_image\plot_7_1day')

In [ ]:

plt.title('CRPS Skill Score')
l1=plt.plot(x,np.array(a_crpsss_my),label='Ours')
l2=plt.plot(x,np.array(a_crpsss_qm),label='QM')
#plt.savefig(r'C:\Users\123\Desktop\crps_image\plot')
h=plt.axhline(y=0, color='r',linestyle='-')
plt.xlabel('days')
plt.ylabel('crps')
plt.legend()
#plt.savefig(r'C:\Users\123\Desktop\crps_image\plot_30_1day')